In [39]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import soundfile

import torch
import torch.utils.data as tud
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torchaudio.transforms import MelSpectrogram, ComputeDeltas

from torch.optim.adamw import AdamW


import re
import json

import os
from os import listdir
from os.path import isfile, join

import math
import random

from pathlib import Path
from IPython.display import Audio

from tqdm import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt

import pyaudio
import wave

from fastprogress import master_bar, progress_bar

import pandas as pd 
import os

from scipy.io import wavfile
from scipy import signal
import tqdm
import IPython
import re

import whisper_timestamped as whisper
import librosa

from torch.utils.data import DataLoader
import ast

import glob
import difflib

In [2]:
wake_words = [
    "fuck"
    ,"fucking"
    ,"ass"
    ,"bitch"
    ,"dick"
    ,"fag"
    ,"faggot"
    ,"shit"
    ,"cunt"
    ,"whore"
    ,"cock"
    ,"pussy"
    ,"cocksucker"

]

wake_words_sequence = ["0", "1", "2","3","4","5","6","7","8","9","10","11","12","13"]


In [3]:
wake_word_seq_map = dict(zip(wake_words, wake_words_sequence))
regex_pattern = r"\b(?:{})\b".format("|".join(map(re.escape, wake_words)))
pattern = re.compile(regex_pattern, flags=re.IGNORECASE)


def wake_words_search(pattern, word):
    try:
        return bool(pattern.search(word))
    except TypeError:
        return False

In [4]:
positive_train_data = pd.read_csv('dataset/cuss_word_time_stamp_wisper.csv')

negative_train_data = pd.read_csv('dataset/cuss_word_time_stamp_wisper_false.csv')
combined_train_data = pd.concat([positive_train_data, negative_train_data])
combined_train_data

,path,sentence,timestamps,flag
0,en-US_004NOMVGP0_85153_True.wav,it's called ur mom being fucked,"{""It's"": {'start': 0.06, 'end': 0.3}, 'called'...",True
1,en-US_008A2TOMAT_196782_True.wav,son of a fucking whore,"{'Son': {'start': 0.1, 'end': 0.32}, 'of': {'s...",True
2,en-US_008E2KYNNP_131758_True.wav,look at it go motherfucker,"{'Look': {'start': 0.28, 'end': 0.5}, 'at': {'...",True
3,en-US_008KASX3NT_46781_True.wav,i hate this pinballl shit,"{'I': {'start': 0.06, 'end': 0.26}, 'hate': {'...",True
4,en-US_00EDDNIP6X_127419_True.wav,very noob motherfucker,"{'Very': {'start': 0.1, 'end': 0.4}, 'new': {'...",True
...,...,...,...,...
16245,en-US_2UTD6TT58B_167027_False.wav,why are you guys fighting,"{'Why': {'start': 0.06, 'end': 0.24}, 'are': {...",False
16246,en-US_2UTDVLG2V8_122831_False.wav,crobelus with lothars,"{'Crabalus': {'start': 0.08, 'end': 0.82}, 'wi...",False
16247,en-US_2UTGOUHX8I_156341_False.wav,what are you buying mk,"{'What': {'start': 0.1, 'end': 0.24}, 'are': {...",False
16248,en-US_2UTLOP2E59_23736_False.wav,"legend is legend, stay in noob forever","{'Legend': {'start': 0.06, 'end': 0.48}, 'is':...",False


In [5]:
positive_train_data = pd.read_csv('dataset/cuss_word_time_stamp_wisper.csv')

negative_train_data = pd.read_csv('dataset/cuss_word_time_stamp_wisper_false.csv')

negative_train_data['timestamps'].apply(lambda x: {})

combined_train_data = pd.concat([positive_train_data, negative_train_data])


combined_train_data['timestamps'] = combined_train_data['timestamps'].apply(ast.literal_eval)
combined_train_data['timestamps'] = combined_train_data['timestamps'].apply(lambda x: {re.sub('[^\w\s]', '', key): value for key, value in x.items()})


In [127]:
test_ds = pd.read_csv('dataset/test.csv')
test_ds['timestamps'] = test_ds['timestamps'].apply(ast.literal_eval)
test_ds['timestamps'] = test_ds['timestamps'].apply(lambda x: {re.sub('[^\w\s]', '', key): value for key, value in x.items()})


In [6]:
def similar(a,b):
    seq = difflib.SequenceMatcher(None,a,b)
    return seq.ratio()

In [7]:
data_path = 'dataset/Data/'

In [173]:
key_pattern = re.compile(r"'(?P<key>[^']+)'")

def compute_labels(metadata, audio_data):
    label = 0 # by default negative label

    # if it is generated data then 
    if metadata['sentence'].lower() in wake_words:
        label = int(wake_word_seq_map[metadata['sentence'].lower()])
    else:
        # if the sentence has one wakeword get label and end timestamp
        wake_word_found = False
        for word in metadata['sentence'].lower().split():
            word = re.sub('\W+', '', word)
            for wake_word in wake_words:
                if similar(word, wake_word.lower()) >= 0.8 or word in wake_word.lower() or wake_word.lower() in word.lower(): 
                    wake_word_found = True

                    label = 1
                    if word in  metadata['timestamps']:
                        timestamps = metadata['timestamps']
                        if type(timestamps) == str:
                          #timestamps = json.loads(key_pattern.sub(r'"\g"', timestamps))
                            timestamps = eval(timestamps)
                        word_ts = timestamps[word]
                        audio_start_idx = int((word_ts['start'] * 1000) * sr / 1000)
                        audio_end_idx = int((word_ts['end'] * 1000) * sr / 1000)
                        audio_data = audio_data[audio_start_idx:audio_end_idx]
                    else: # if there are issues with word alignment, we might not get ts
                        label = 0 
            if label == 1:
                break
    return label, audio_data



class AudioCollator(object):
  def __init__(self, noise_set=None):
    self.noise_set = noise_set

  def __call__(self, batch):
    batch_tensor = {}
    window_size_ms = 750
    max_length = int(window_size_ms/1000 * sr)
    audio_tensors = []
    labels = []
    for sample in batch:
      # get audio_data in tensor format
      audio_data = librosa.core.load(data_path+sample['path'], sr=sr, mono=True)[0]
      # get the label and its audio
      label, audio_data = compute_labels(sample, audio_data)
      audio_data_length = audio_data.size / sr * 1000 #ms
  
      # below is to make sure that we always got length of 12000 
      # i.e 750 ms with sr 16000
      # trim to max_length
      if audio_data_length > window_size_ms:
        # randomly trim either at start and end
        if random.random() < 0.5:
          audio_data = audio_data[:max_length]
        else:
          audio_data = audio_data[audio_data.size-max_length:]
  
      # pad with zeros
      if audio_data_length < window_size_ms:
        # randomly either append or prepend
        if random.random() < 0.5:
          audio_data = np.append(audio_data, np.zeros(int(max_length - audio_data.size)))
        else:
          audio_data = np.append(np.zeros(int(max_length - audio_data.size)), audio_data)

      # Add noise
      if self.noise_set:
        noise_level =  random.randint(1, 5)/10 # 10 to 50%
        noise_sample = librosa.core.load(self.noise_set[random.randint(0,len(self.noise_set)-1)], sr=sr, mono=True)[0]
        # randomly select first or last seq of noise
        if random.random() < 0.5:
          audio_data = (1 - noise_level) * audio_data +  noise_level * noise_sample[:max_length]
        else:
          audio_data = (1 - noise_level) * audio_data +  noise_level * noise_sample[-max_length:]
  
      audio_tensors.append(torch.from_numpy(audio_data))
      labels.append(label)
  
    batch_tensor = {
        'audio': torch.stack(audio_tensors),
        'labels': torch.tensor(labels) 
    }
    
    return batch_tensor
     

In [122]:
train_ds = combined_train_data

In [87]:
hey_pattern = re.compile(r'\bfuck\b', flags=re.IGNORECASE)
fourth_pattern = re.compile(r'\bass\b', flags=re.IGNORECASE)
brain_pattern = re.compile(r'\bfucking\b', flags=re.IGNORECASE)

print(f"Total fuck word {(train_ds[[wake_words_search(hey_pattern, sentence) for sentence in train_ds['sentence']]].size/train_ds.size) * 100} %")
print(f"Total ass word {(train_ds[[wake_words_search(fourth_pattern, sentence) for sentence in train_ds['sentence']]].size/train_ds.size) * 100} %")
print(f"Total fucking word {(train_ds[[wake_words_search(brain_pattern, sentence) for sentence in train_ds['sentence']]].size/train_ds.size) * 100} %")



Total fuck word 3.915496542654732 %
Total ass word 1.8453315233529883 %
Total fucking word 5.039664022398506 %


In [88]:
noise_test = glob.glob('dataset/MS-SNSD/noise_test/*.wav')
noise_train = glob.glob('dataset/MS-SNSD/noise_train/*.wav')


In [89]:
test_ds

,path,sentence,timestamps,flag
0,en-US_00265106HO_29664_False.wav,stupid shit dont know how to deward,"{'Stupid': {'start': 0.08, 'end': 0.42}, 'shit...",False
1,en-US_002OLMQ4PK_209573_False.wav,report my fucking team,"{'Report': {'start': 0.06, 'end': 0.44}, 'my':...",False
2,en-US_004406IY83_103288_False.wav,you can go fuck yourself and your triple hard,"{'You': {'start': 0.06, 'end': 0.16}, 'can': {...",False
3,en-US_004G2PW5WV_11707_False.wav,who the fuck counter picks treant,"{'Who': {'start': 0.04, 'end': 0.24}, 'the': {...",False
4,en-US_005G6PKSZO_18276_False.wav,watch this faggot feed again,"{'Watch': {'start': 0.36, 'end': 0.68}, 'this'...",False
...,...,...,...,...
14978,en-US_ZZEMZI049J_145404_False.wav,and this fucking game is lugging,"{'and': {'start': 0.08, 'end': 0.24}, 'this': ...",False
14979,en-US_ZZHXHE7GFJ_174842_False.wav,dont fuck me in offlane,"{""Don't"": {'start': 0.28, 'end': 0.58}, 'fuck'...",False
14980,en-US_ZZNT61F3FJ_7645_False.wav,bristle the fuck u gonna do lol,"{'Bristle': {'start': 0.1, 'end': 0.6}, 'the':...",False
14981,en-US_ZZPMV3TLUB_68367_False.wav,like its the first fucking doto match,"{'Like': {'start': 0.04, 'end': 0.24}, ""it's"":...",False


In [128]:
batch_size = 64
num_workers = 0

train_audio_collator = AudioCollator(noise_set=noise_train)
train_dl = tud.DataLoader(test_ds.to_dict(orient='records'),
                  batch_size=1,
                  drop_last=True,
                  shuffle=True,
                  num_workers=num_workers,collate_fn=train_audio_collator)


In [165]:
for data in train_dl:
    print(data)

{'audio': tensor([[-6.1467e-09,  9.7416e-08, -8.3819e-08,  ..., -1.6374e-03,
         -1.2047e-03, -4.6460e-04]], dtype=torch.float64), 'labels': tensor([1])}
{'audio': tensor([[0.0434, 0.0777, 0.0873,  ..., 0.1669, 0.1675, 0.1473]]), 'labels': tensor([0])}
{'audio': tensor([[-0.0002, -0.0005, -0.0005,  ...,  0.0366,  0.0407, -0.0189]]), 'labels': tensor([0])}
{'audio': tensor([[-0.0093, -0.0076, -0.0079,  ...,  0.0000,  0.0000,  0.0000]],
       dtype=torch.float64), 'labels': tensor([1])}
{'audio': tensor([[-3.1395e-05,  1.1468e-04, -7.5544e-05,  ...,  1.5917e-02,
          1.3242e-02,  1.2719e-02]], dtype=torch.float64), 'labels': tensor([1])}
{'audio': tensor([[ 0.0318,  0.0443, -0.0471,  ..., -0.0101, -0.0115, -0.0095]]), 'labels': tensor([0])}
{'audio': tensor([[0.0664, 0.0908, 0.1048,  ..., 0.0795, 0.0781, 0.0529]]), 'labels': tensor([0])}
{'audio': tensor([[ 0.0005,  0.0005,  0.0002,  ...,  0.0003, -0.0007, -0.0019]],
       dtype=torch.float64), 'labels': tensor([1])}
{'audio'

{'audio': tensor([[-0.0008, -0.0008,  0.0004,  ..., -0.0041, -0.0345,  0.0197]],
       dtype=torch.float64), 'labels': tensor([1])}
{'audio': tensor([[-0.1001, -0.1376, -0.1210,  ...,  0.0344,  0.0488,  0.0586]],
       dtype=torch.float64), 'labels': tensor([1])}
{'audio': tensor([[-0.2602, -0.1444,  0.0319,  ...,  0.0431,  0.0610,  0.0732]],
       dtype=torch.float64), 'labels': tensor([1])}
{'audio': tensor([[0.0416, 0.0355, 0.0298,  ..., 0.1793, 0.1936, 0.2144]],
       dtype=torch.float64), 'labels': tensor([1])}
{'audio': tensor([[ 1.1389e-03,  1.3657e-03,  1.4949e-03,  ..., -7.0563e-07,
         -3.2543e-06, -5.7685e-06]], dtype=torch.float64), 'labels': tensor([1])}
{'audio': tensor([[-0.0302, -0.0268, -0.0145,  ..., -0.0543, -0.0455, -0.0404]],
       dtype=torch.float64), 'labels': tensor([1])}
{'audio': tensor([[0.0007, 0.0007, 0.0007,  ..., 0.0708, 0.0301, 0.0169]]), 'labels': tensor([0])}
{'audio': tensor([[-1.3011e-05, -5.6291e-06,  4.3781e-06,  ...,  5.1651e-02,
      

KeyboardInterrupt: 

In [174]:
for key, sample in test_ds.iterrows():
    audio_data = librosa.core.load(data_path+sample['path'], sr=sr, mono=True)[0]
    print(sample['timestamps']
    label, audio_data = compute_labels(sample, audio_data)
    print(label)
    

{'Stupid': {'start': 0.08, 'end': 0.42}, 'shit': {'start': 0.42, 'end': 0.66}, 'dont': {'start': 0.66, 'end': 0.84}, 'know': {'start': 0.84, 'end': 1.02}, 'how': {'start': 1.02, 'end': 1.16}, 'to': {'start': 1.16, 'end': 1.32}, 'do': {'start': 1.32, 'end': 1.44}, 'it': {'start': 1.44, 'end': 1.64}}
1
{'Report': {'start': 0.06, 'end': 0.44}, 'my': {'start': 0.44, 'end': 0.66}, 'fucking': {'start': 0.66, 'end': 0.98}, 'team': {'start': 0.98, 'end': 1.34}}
1
{'You': {'start': 0.06, 'end': 0.16}, 'can': {'start': 0.16, 'end': 0.34}, 'go': {'start': 0.34, 'end': 0.54}, 'fuck': {'start': 0.54, 'end': 0.72}, 'yourself': {'start': 0.72, 'end': 1.12}, 'in': {'start': 1.12, 'end': 1.28}, 'your': {'start': 1.28, 'end': 1.44}, 'triplehard': {'start': 1.44, 'end': 2.04}}
1
{'Who': {'start': 0.04, 'end': 0.24}, 'the': {'start': 0.24, 'end': 0.48}, 'fuck': {'start': 0.48, 'end': 0.78}, 'counter': {'start': 0.78, 'end': 1.18}, 'picks': {'start': 1.18, 'end': 1.6}, 'trend': {'start': 1.6, 'end': 2.02}}

{'Who': {'start': 0.06, 'end': 0.24}, 'fuck': {'start': 0.24, 'end': 0.44}, 'your': {'start': 0.44, 'end': 0.62}, 'step': {'start': 0.62, 'end': 0.84}, 'mom': {'start': 0.84, 'end': 1.1}}
1
{'Such': {'start': 0.12, 'end': 0.36}, 'a': {'start': 0.36, 'end': 0.58}, 'fucking': {'start': 0.58, 'end': 0.96}, 'bad': {'start': 0.96, 'end': 1.24}, 'carry': {'start': 1.24, 'end': 1.66}, 'and': {'start': 1.66, 'end': 1.94}, 'midplayer': {'start': 1.94, 'end': 2.56}}
1
{'Shut': {'start': 0.08, 'end': 0.32}, 'the': {'start': 0.32, 'end': 0.52}, 'fuck': {'start': 0.52, 'end': 0.7}, 'up': {'start': 0.7, 'end': 0.94}, 'and': {'start': 0.94, 'end': 1.16}, 'thank': {'start': 1.16, 'end': 1.38}, 'you': {'start': 1.38, 'end': 1.48}, 'Nigma': {'start': 1.56, 'end': 1.8}}
1
{'Shut': {'start': 0.1, 'end': 0.34}, 'the': {'start': 0.34, 'end': 0.48}, 'fuck': {'start': 0.48, 'end': 0.66}, 'up': {'start': 0.66, 'end': 0.86}, 'retard': {'start': 0.86, 'end': 1.16}}
1
{'Take': {'start': 0.3, 'end': 0.54}, 'the': 

{'Where': {'start': 0.06, 'end': 0.3}, 'is': {'start': 0.3, 'end': 0.58}, 'Fairy': {'start': 0.58, 'end': 0.92}, 'Fires': {'start': 0.92, 'end': 1.44}, 'fucking': {'start': 1.44, 'end': 1.82}, 'Encha': {'start': 1.82, 'end': 2.3}}
1
{'Fuck': {'start': 0.28, 'end': 0.62}, 'this': {'start': 0.62, 'end': 0.88}, 'solo': {'start': 0.88, 'end': 1.24}, 'support': {'start': 1.24, 'end': 1.8}}
0
{'How': {'start': 0.38, 'end': 0.62}, 'the': {'start': 0.62, 'end': 0.8}, 'fuck': {'start': 0.8, 'end': 1.08}, 'Slark': {'start': 1.08, 'end': 1.5}, 'hasnt': {'start': 1.5, 'end': 2.02}, 'BB': {'start': 2.02, 'end': 2.48}}
1
{'You': {'start': 0.42, 'end': 0.66}, 'start': {'start': 0.66, 'end': 0.88}, 'the': {'start': 0.88, 'end': 1.04}, 'shit': {'start': 1.04, 'end': 1.28}, 'talk': {'start': 1.28, 'end': 1.54}}
1
{'You': {'start': 0.06, 'end': 0.22}, 'made': {'start': 0.22, 'end': 0.42}, 'some': {'start': 0.42, 'end': 0.66}, 'fucking': {'start': 0.66, 'end': 1.0}, 'god': {'start': 1.0, 'end': 1.26}, 'aw

{'Feptogame': {'start': 0.08, 'end': 0.7}, 'in': {'start': 0.7, 'end': 0.86}, 'your': {'start': 0.86, 'end': 1.0}, 'mother': {'start': 1.0, 'end': 1.3}}
0
{'Fuck': {'start': 0.24, 'end': 0.48}, 'you': {'start': 0.48, 'end': 0.66}, 'with': {'start': 0.66, 'end': 0.78}, 'that': {'start': 0.78, 'end': 0.96}, 'broken': {'start': 0.96, 'end': 1.22}, 'hero': {'start': 1.22, 'end': 1.58}}
0
{'What': {'start': 0.04, 'end': 0.18}, 'a': {'start': 0.18, 'end': 0.34}, 'fucking': {'start': 0.34, 'end': 0.64}, 'boring': {'start': 0.64, 'end': 0.94}, 'hero': {'start': 0.94, 'end': 1.28}}
1
{'Are': {'start': 0.06, 'end': 0.16}, 'you': {'start': 0.16, 'end': 0.36}, 'fucking': {'start': 0.36, 'end': 0.7}, 'joking': {'start': 0.7, 'end': 1.06}}
1
{'Now': {'start': 0.06, 'end': 0.38}, 'Pudge': {'start': 0.38, 'end': 0.76}, 'will': {'start': 0.76, 'end': 0.98}, 'fuck': {'start': 0.98, 'end': 1.18}, 'you': {'start': 1.18, 'end': 1.4}}
1
{'Marana': {'start': 0.08, 'end': 0.48}, 'MVP': {'start': 0.48, 'end': 

{'Just': {'start': 0.12, 'end': 0.44}, 'W8': {'start': 0.44, 'end': 1.1}, 'fucking': {'start': 1.1, 'end': 1.42}, 'paper': {'start': 1.42, 'end': 1.74}}
1
{'and': {'start': 0.08, 'end': 0.3}, 'fucking': {'start': 0.3, 'end': 0.66}, 'polish': {'start': 0.66, 'end': 1.1}, 'cancer': {'start': 1.1, 'end': 1.58}}
1
{'Just': {'start': 0.22, 'end': 0.48}, 'fucking': {'start': 0.48, 'end': 0.82}, 'stay': {'start': 0.82, 'end': 1.06}, 'away': {'start': 1.06, 'end': 1.44}}
1
{'Now': {'start': 0.08, 'end': 0.38}, 'this': {'start': 0.38, 'end': 0.7}, 'fucking': {'start': 0.7, 'end': 1.08}, 'strong': {'start': 1.08, 'end': 1.56}, 'wanna': {'start': 1.56, 'end': 1.92}, 'death': {'start': 1.92, 'end': 2.32}}
1
{'You': {'start': 0.3, 'end': 0.56}, 'worthless': {'start': 0.56, 'end': 1.08}, 'bag': {'start': 1.08, 'end': 1.48}, 'at': {'start': 1.48, 'end': 1.68}, 'Lear': {'start': 1.68, 'end': 1.98}}
0
{'How': {'start': 0.08, 'end': 0.3}, 'the': {'start': 0.3, 'end': 0.5}, 'fuck': {'start': 0.5, 'end': 

...

{'Fair': {'start': 0.14, 'end': 0.44}, 'and': {'start': 0.44, 'end': 0.68}, 'fucking': {'start': 0.68, 'end': 1.0}, 'balanced': {'start': 1.0, 'end': 1.48}}
1
{'Sven': {'start': 0.12, 'end': 0.36}, 'only': {'start': 0.36, 'end': 0.7}, 'won': {'start': 0.7, 'end': 0.94}, 'giving': {'start': 0.94, 'end': 1.26}, 'fuck': {'start': 1.26, 'end': 1.52}, 'a': {'start': 1.52, 'end': 1.62}, 'bowo': {'start': 1.62, 'end': 1.92}, 'this': {'start': 1.92, 'end': 2.14}, 'game': {'start': 2.14, 'end': 2.38}}
1
{'Defend': {'start': 0.08, 'end': 0.6}, 'this': {'start': 2.76, 'end': 2.98}, 'USLE': {'start': 0.94, 'end': 1.74}, 'SSS': {'start': 1.74, 'end': 2.64}, 'in': {'start': 2.64, 'end': 2.76}, 'contest': {'start': 2.98, 'end': 3.58}}
0
{'Embers': {'start': 0.08, 'end': 0.68}, 'so': {'start': 0.68, 'end': 0.98}, 'fucking': {'start': 0.98, 'end': 1.36}, 'dumb': {'start': 1.36, 'end': 1.78}}
1
{'I': {'start': 0.06, 'end': 0.18}, 'dont': {'start': 0.18, 'end': 0.44}, 'fucking': {'start': 0.44, 'end': 0.

{'IDK': {'start': 0.5, 'end': 1.1}, 'how': {'start': 1.1, 'end': 1.26}, 'that': {'start': 1.26, 'end': 1.46}, 'shit': {'start': 1.46, 'end': 1.66}, 'works': {'start': 1.66, 'end': 1.92}, 'main': {'start': 1.92, 'end': 2.22}}
1
{'Veno': {'start': 0.36, 'end': 0.8}, 'please': {'start': 0.94, 'end': 1.14}, 'get': {'start': 1.14, 'end': 1.36}, 'the': {'start': 1.36, 'end': 1.48}, 'fuck': {'start': 1.48, 'end': 1.66}, 'out': {'start': 1.66, 'end': 2.0}}
1
{'Stop': {'start': 0.36, 'end': 0.7}, 'pausing': {'start': 0.7, 'end': 1.16}, 'you': {'start': 1.16, 'end': 1.48}, 'fucking': {'start': 1.48, 'end': 1.82}, 'ended': {'start': 1.82, 'end': 2.18}}
0
{'Pussy': {'start': 0.06, 'end': 0.36}, 'Blade': {'start': 0.36, 'end': 0.64}, 'for': {'start': 0.64, 'end': 0.86}, 'everyone': {'start': 0.86, 'end': 1.26}}
0
{'and': {'start': 0.1, 'end': 0.34}, 'Luna': {'start': 0.34, 'end': 0.64}, 'with': {'start': 0.64, 'end': 0.98}, 'no': {'start': 0.98, 'end': 1.24}, 'fucking': {'start': 1.24, 'end': 1.68}

{'Just': {'start': 0.06, 'end': 0.34}, 'report': {'start': 0.34, 'end': 0.78}, 'fucking': {'start': 0.78, 'end': 1.18}, 'paw': {'start': 1.18, 'end': 1.52}}
1
{'You': {'start': 0.06, 'end': 0.34}, 'were': {'start': 0.34, 'end': 0.64}, 'so': {'start': 0.64, 'end': 0.92}, 'fucking': {'start': 0.92, 'end': 1.28}, 'toxic': {'start': 1.28, 'end': 1.8}}
1
{'Fucking': {'start': 0.4, 'end': 0.68}, 'told': {'start': 0.68, 'end': 0.96}, 'him': {'start': 0.96, 'end': 1.1}, 'I': {'start': 1.1, 'end': 1.18}, 'had': {'start': 1.18, 'end': 1.36}, 'long': {'start': 1.36, 'end': 1.62}}
0
{'We': {'start': 0.3, 'end': 0.44}, 'have': {'start': 0.44, 'end': 0.64}, 'a': {'start': 0.64, 'end': 0.7}, 'fucking': {'start': 0.7, 'end': 1.02}, 'GLPHY': {'start': 1.02, 'end': 2.2}}
1
{'How': {'start': 0.16, 'end': 0.36}, 'fucking': {'start': 0.36, 'end': 0.64}, 'pathetic': {'start': 0.64, 'end': 1.04}, 'can': {'start': 1.04, 'end': 1.28}, 'you': {'start': 1.28, 'end': 1.42}, 'get': {'start': 1.42, 'end': 1.64}}
1


{'Shut': {'start': 0.06, 'end': 0.32}, 'the': {'start': 0.32, 'end': 0.5}, 'fuck': {'start': 0.5, 'end': 0.72}, 'up': {'start': 0.72, 'end': 0.96}, 'fucking': {'start': 2.36, 'end': 2.64}, 'Korean': {'start': 1.28, 'end': 1.68}, 'shit': {'start': 1.68, 'end': 2.0}, 'to': {'start': 2.0, 'end': 2.2}, 'your': {'start': 2.2, 'end': 2.36}, 'opaoba': {'start': 2.64, 'end': 3.26}}
1
{'Zero': {'start': 0.06, 'end': 0.4}, 'supports': {'start': 0.4, 'end': 1.04}, 'in': {'start': 1.04, 'end': 1.22}, 'this': {'start': 1.22, 'end': 1.44}, 'shit': {'start': 1.44, 'end': 1.7}, 'team': {'start': 1.7, 'end': 1.98}}
1
{'Dont': {'start': 0.08, 'end': 0.5}, 'fucking': {'start': 0.5, 'end': 0.88}, 'tell': {'start': 0.88, 'end': 1.1}, 'me': {'start': 1.1, 'end': 1.26}, 'youre': {'start': 1.26, 'end': 1.48}, 'waiting': {'start': 1.48, 'end': 1.8}, 'for': {'start': 1.8, 'end': 2.12}, 'EGIS': {'start': 2.12, 'end': 2.64}, 'to': {'start': 2.64, 'end': 2.78}, 'expire': {'start': 2.78, 'end': 3.36}}
1
{'Holy': {'

KeyboardInterrupt: 

In [172]:
timestamps = {'Fuck': {'start': 0.08, 'end': 0.3}, 'you': {'start': 0.3, 'end': 0.5}, 'guys': {'start': 0.5, 'end': 0.76}, 'for': {'start': 0.76, 'end': 0.94}, 'picking': {'start': 0.94, 'end': 1.22}, 'antipun': {'start': 1.22, 'end': 1.9}, 'heroes': {'start': 1.9, 'end': 2.3}}


In [160]:
metadata = sample
label = 0 # by default negative label

# if it is generated data then 
if metadata['sentence'].lower() in wake_words:
    label = int(wake_word_seq_map[metadata['sentence'].lower()])
else:
    # if the sentence has one wakeword get label and end timestamp
    wake_word_found = False
    for word in metadata['sentence'].lower().split():
        word = re.sub('\W+', '', word)
        for wake_word in wake_words:
            if similar(word, wake_word) >= 0.8 or word in wake_word or wake_word in word: 
                wake_word_found = True
                print(True)

                label = 1
                print(word in metadata['timestamps'])
                if word in  metadata['timestamps']:
                    timestamps = metadata['timestamps']
                    if type(timestamps) == str:
                      #timestamps = json.loads(key_pattern.sub(r'"\g"', timestamps))
                        timestamps = eval(timestamps)
                    word_ts = timestamps[word]
                    audio_start_idx = int((word_ts['start'] * 1000) * sr / 1000)
                    audio_end_idx = int((word_ts['end'] * 1000) * sr / 1000)
                    audio_data = audio_data[audio_start_idx:audio_end_idx]
                else: # if there are issues with word alignment, we might not get ts
                    label = 0 
        if label == 1:
            break
                
            
            
        

True
True


In [161]:
word

'shit'

In [126]:
eval(sample['timestamps'])

{'Report': {'start': 0.06, 'end': 0.44},
 'my': {'start': 0.44, 'end': 0.66},
 'fucking': {'start': 0.66, 'end': 0.98},
 'team.': {'start': 0.98, 'end': 1.34}}

In [110]:
sample['timestamps']

"{'Report': {'start': 0.06, 'end': 0.44}, 'my': {'start': 0.44, 'end': 0.66}, 'fucking': {'start': 0.66, 'end': 0.98}, 'team.': {'start': 0.98, 'end': 1.34}}"

In [117]:
eval(timestamps)


{'Report': {'start': 0.06, 'end': 0.44},
 'my': {'start': 0.44, 'end': 0.66},
 'fucking': {'start': 0.66, 'end': 0.98},
 'team.': {'start': 0.98, 'end': 1.34}}

In [118]:
timestamps = timestamps.replace('"', "'") 

timestamps = json.loads(eval(timestamps))


TypeError: the JSON object must be str, bytes or bytearray, not dict

In [27]:
sr = 24000

num_mels = 40 # https://en.wikipedia.org/wiki/Mel_scale
num_fft = 512 # window length - Fast Fourier Transform
hop_length = 200  # making hops of size hop_length each time to sample the next window

def audio_transform(audio_data):
  # Transformations
  # Mel-scale spectrogram is a combination of Spectrogram and mel scale conversion
  # 1. compute FFT - for each window to transform from time domain to frequency domain
  # 2. Generate Mel Scale - Take entire freq spectrum & seperate to n_mels evenly spaced
  #    frequencies. (not by distance on freq domain but distance as it is heard by human ear)
  # 3. Generate Spectrogram - For each window, decompose the magnitude of the signal
  #    into its components, corresponding to the frequencies in the mel scale. 
  mel_spectrogram  = MelSpectrogram(n_mels=num_mels,
                                    sample_rate=sr,
                                    n_fft=num_fft,
                                    hop_length=hop_length,
                                    norm='slaney')
  mel_spectrogram.to(device)
  log_mels = mel_spectrogram(audio_data.float()).add_(1e-7).log_().contiguous()
  # returns (channel, n_mels, time) 
  return log_mels.to(device)
     


In [28]:
from typing import Iterable
class ZmuvTransform(nn.Module):
    def __init__(self):
        super().__init__()
        self.register_buffer('total', torch.zeros(1))
        self.register_buffer('mean', torch.zeros(1))
        self.register_buffer('mean2', torch.zeros(1))

    def update(self, data, mask=None):
        with torch.no_grad():
            if mask is not None:
                data = data * mask
                mask_size = mask.sum().item()
            else:
                mask_size = data.numel()
            self.mean = (data.sum() + self.mean * self.total) / (self.total + mask_size)
            self.mean2 = ((data ** 2).sum() + self.mean2 * self.total) / (self.total + mask_size)
            self.total += mask_size

    def initialize(self, iterable: Iterable[torch.Tensor]):
        for ex in iterable:
            self.update(ex)

    @property
    def std(self):
        return (self.mean2 - self.mean ** 2).sqrt()

    def forward(self, x):
        return (x - self.mean) / self.std


In [29]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [31]:
zmuv_audio_collator = AudioCollator()
zmuv_dl = tud.DataLoader(train_ds.to_dict(orient='records'),
                  batch_size=1,
                  num_workers=num_workers,
                  collate_fn=zmuv_audio_collator)

zmuv_transform = ZmuvTransform().to(device)
if Path("zmuv.pt.bin").exists():
    zmuv_transform.load_state_dict(torch.load(str("zmuv.pt.bin")))
else:
  for idx, batch in enumerate(zmuv_dl):
    zmuv_transform.update(batch['audio'].to(device))
  print(dict(zmuv_mean=zmuv_transform.mean, zmuv_std=zmuv_transform.std))
  torch.save(zmuv_transform.state_dict(), str("zmuv.pt.bin"))

print(f"Mean is {zmuv_transform.mean.item():0.6f}")
print(f"Standard Deviation is {zmuv_transform.std.item():0.6f}")


Mean is 0.000012
Standard Deviation is 0.104256


In [32]:
class CNN(nn.Module):
    def __init__(self, num_labels, num_maps1, num_maps2, num_hidden_input, hidden_size):
        super(CNN, self).__init__()
        conv0 = nn.Conv2d(1, num_maps1, (8, 16), padding=(4, 0), stride=(2, 2), bias=True)
        pool = nn.MaxPool2d(2)
        conv1 = nn.Conv2d(num_maps1, num_maps2, (5, 5), padding=2, stride=(2, 1), bias=True)
        self.num_hidden_input = num_hidden_input
        self.encoder1 = nn.Sequential(conv0,
                                      nn.ReLU(),
                                      pool,
                                      nn.BatchNorm2d(num_maps1, affine=True))
        self.encoder2 = nn.Sequential(conv1,
                                      nn.ReLU(),
                                      pool,
                                      nn.BatchNorm2d(num_maps2, affine=True))
        self.output = nn.Sequential(nn.Linear(num_hidden_input, hidden_size),
                                    nn.ReLU(),
                                    nn.Dropout(0.1),
                                    nn.Linear(hidden_size, num_labels))
        
    def forward(self, x):
        x = x.permute(0, 1, 3, 2)  # change to (time, n_mels)
        # pass through first conv layer
        x1 = self.encoder1(x)
        # pass through second conv layer
        x2 = self.encoder2(x1)
        # flattening - keep first dim batch same, flatten last 3 dims
        x = x2.view(x2.size(0), -1)
        return self.output(x)

In [33]:
num_labels = 2 # oov
num_maps1  = 48
num_maps2  = 64
num_hidden_input =  1152
hidden_size = 128
model = CNN(num_labels, num_maps1, num_maps2, num_hidden_input, hidden_size)


In [34]:
model.to(device)
model

CNN(
  (encoder1): Sequential(
    (0): Conv2d(1, 48, kernel_size=(8, 16), stride=(2, 2), padding=(4, 0))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (encoder2): Sequential(
    (0): Conv2d(48, 64, kernel_size=(5, 5), stride=(2, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (output): Sequential(
    (0): Linear(in_features=1152, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=128, out_features=2, bias=True)
  )
)

In [35]:
summary(model.to('cuda'), input_size=(1,40,91))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 48, 46, 13]           6,192
              ReLU-2           [-1, 48, 46, 13]               0
         MaxPool2d-3            [-1, 48, 23, 6]               0
         MaxPool2d-4            [-1, 48, 23, 6]               0
       BatchNorm2d-5            [-1, 48, 23, 6]              96
            Conv2d-6            [-1, 64, 12, 6]          76,864
              ReLU-7            [-1, 64, 12, 6]               0
         MaxPool2d-8             [-1, 64, 6, 3]               0
         MaxPool2d-9             [-1, 64, 6, 3]               0
      BatchNorm2d-10             [-1, 64, 6, 3]             128
           Linear-11                  [-1, 128]         147,584
             ReLU-12                  [-1, 128]               0
          Dropout-13                  [-1, 128]               0
           Linear-14                   

In [36]:
learning_rate = 0.001
weight_decay = 0.0001 # Weight regularization
lr_decay = 0.95

criterion = nn.CrossEntropyLoss()
params = list(filter(lambda x: x.requires_grad, model.parameters()))

optimizer = AdamW(model.parameters(), learning_rate, weight_decay=weight_decay)

In [37]:
def train():
    epochs = 20

    # config for progress bar
    mb = master_bar(range(epochs))
    mb.names = ['Training loss', 'Validation loss']
    x = []

    training_losses = []
    validation_losses = []

    valid_mean_min = np.Inf

    for epoch in mb:
      x.append(epoch)
      # Evaluate
      model.train()
      total_loss = torch.Tensor([0.0]).to(device)
      #pbar = tqdm(train_dl, total=len(train_dl), position=0, desc="Training", leave=True)
      for batch in progress_bar(train_dl, parent=mb):
        audio_data = batch['audio'].to(device)
        labels = batch['labels'].to(device)
        # get mel spectograms
        mel_audio_data = audio_transform(audio_data)
        # do zmuv transform
        mel_audio_data = zmuv_transform(mel_audio_data)
        predicted_scores = model(mel_audio_data.unsqueeze(1))
        # get loss
        loss = criterion(predicted_scores, labels.long())

        optimizer.zero_grad()
        model.zero_grad()

        # backward propagation
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            total_loss += loss

      for group in optimizer.param_groups:
        group["lr"] *= lr_decay

      mean = total_loss / len(train_dl)
      training_losses.append(mean.cpu())
      return model,training_losses
    

In [42]:
if __name__ == '__main__':
    from multiprocessing import freeze_support
    freeze_support()
    model,training_losses = train()


KeyboardInterrupt: 

In [41]:
training_losses

[]